In [7]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

df = pd.read_csv("glassdoor_dataset/glassdoor.csv")




# Filtering the main glassdoor.csv file, a lot of columns are HTML related elements that are not very relevant for an analytical purpose.

filtered_df = df.loc[:,
                [
                    "gaTrackerData.jobTitle",
                    "gaTrackerData.location",
                    "gaTrackerData.locationType",
                    "gaTrackerData.sector",
                    "header.easyApply",
                    "header.employerName",
                    "header.jobTitle",
                    "header.location", # Location City (most of times)
                    "header.posted", # Date job was posted
                    "header.rating", # Company rating by employees
                    "header.urgencyLabel", # Misleading column name, it actually indicates the presence of the "2019 Glassdoor Best Place to Work" award on the job posting
                    "header.payHigh",
                    "header.payMed",
                    "header.payLow",
                    "header.salarySource",
                    "job.description",
                    "job.jobSource",
                    "map.country", # do something with country_names_2_digit_codes
                    "map.employerName",
                    "map.lat",
                    "map.lng",
                    "map.location",
                    "map.address",
                    "map.postalCode",
                    "overview.foundedYear",
                    "overview.hq",
                    "overview.industry",
                    "overview.revenue",
                    "overview.sector",
                    "overview.size",
                    "overview.stock",
                    "overview.type",
                    "overview.description",
                    "overview.mission",
                    "overview.website",
                    "overview.competitors", # foreign key
                    "rating.ceo.name",
                    "rating.ceoApproval",
                    "rating.recommendToFriend",
                    "rating.starRating",
                    "salary.country.name",
                    "benefits.comments", # foreign key
                    "benefits.highlights", # foreign key
                    "reviews", # foreign key
                    "salary.salaries", # foreign key
                    "wwfu", # foreign key
    ],
]

print(filtered_df.head(80))
print(filtered_df.columns)
print(f"There are {len(filtered_df.columns)} columns")
print(filtered_df.describe())
print(f"There are {filtered_df.describe().shape[1]} columns")

                               gaTrackerData.jobTitle gaTrackerData.location  \
0                  Biogas Project Development Manager                 Warsaw   
1   Quality Manager - Boehringer Ingelheim Healthc...                    NaN   
2      Senior Software Engineer (PHP, Elixir, Python)                 Milano   
3                                 Senior SQA Engineer                Karachi   
4                                    Research Manager              Bengaluru   
..                                                ...                    ...   
75               Senior Android Engineer - Innovation               Auckland   
76                              Project Administrator              Cape Town   
77                                 Software Developer                    Pau   
78  Business Development Manager, Mobile Telecomms...                  Lagos   
79  Research Staff Member / Post-doctoral research...                 Zürich   

   gaTrackerData.locationType       gaT